In [1]:
from mbientlab.metawear import MetaWear, libmetawear, parse_value, create_voidp, create_voidp_int
from mbientlab.metawear.cbindings import *
from ctypes import c_void_p, POINTER, CFUNCTYPE, cast, byref, c_uint, c_ubyte, c_longlong
from threading import Event
from time import sleep

In [3]:
# Initialize device and connect
d = MetaWear('D4:06:BD:85:8F:23')
d.connect()
e = Event()

In [4]:
def processor_created(context, pointer):
    global processor
    processor = pointer
    e.set()

# Function pointer
fn_wrapper = FnVoid_VoidP_VoidP(processor_created)

# Acc and gyro signals
acc = libmetawear.mbl_mw_acc_get_acceleration_data_signal(d.board)
gyro = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(d.board)

# Fuse the signals together
signals = (c_void_p * 1)()
signals[0] = gyro
libmetawear.mbl_mw_dataprocessor_fuser_create(acc, signals, 1, None, fn_wrapper)
e.wait()

True

In [ ]:
# Create a logger
logger = create_voidp(lambda fn: libmetawear.mbl_mw_datasignal_log(processor, None, fn), resource = "acc_gyro_logger")

# Start logger
libmetawear.mbl_mw_logging_start(d.board, 0)
# Turn on the accelerometer
libmetawear.mbl_mw_acc_enable_acceleration_sampling(d.board)
libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling(d.board)
libmetawear.mbl_mw_acc_start(d.board)
libmetawear.mbl_mw_gyro_bmi160_start(d.board)


print("Logging data for 5s")
sleep(5.0)

# Turn off the accelerometer
libmetawear.mbl_mw_acc_stop(d.board)
libmetawear.mbl_mw_gyro_bmi160_stop(d.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(d.board)
libmetawear.mbl_mw_gyro_bmi160_disable_rotation_sampling(d.board)
# Stop logging
libmetawear.mbl_mw_logging_stop(d.board)

print("Downloading data")
# Callback function to handle logger entries as we download them
def progress_update_handler(context, entries_left, total_entries):
    if (entries_left == 0):
        e.set()

# Function pointer and handlers for the logger download (LogDownloadHandler -> in the cbindings)
fn_wrapper2 = FnVoid_VoidP_UInt_UInt(progress_update_handler)
download_handler = LogDownloadHandler(context = None, received_progress_update = fn_wrapper2, received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))
callback = FnVoid_VoidP_DataP(lambda ctx, p: print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p))))

# Stop logger
libmetawear.mbl_mw_logger_subscribe(logger, None, callback)

# Download logger contents
libmetawear.mbl_mw_logging_download(d.board, 0, byref(download_handler))
e.wait()

Exception ignored on calling ctypes callback function: <function create_voidp.<locals>.handler at 0x0000019E2DA6F9C0>
Traceback (most recent call last):
  File "C:\Users\jelte\AppData\Local\Programs\Python\Python311\Lib\site-packages\mbientlab\metawear\__init__.py", line 75, in handler
    result[0] = RuntimeError("Could not create " + (kwargs['resource'] if 'resource' in kwarg else "resource") ) if pointer == None else pointer
                                                                                        ^^^^^
NameError: name 'kwarg' is not defined


In [5]:
d.on_disconnect = lambda status: print ("we are disconnected!")
d.disconnect()

we are disconnected!
